In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
# R.EnableImplicitMT()
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

# Link the C++ fit_gaus_tail function to a TF1 so the fitting of the histogram works properly in Python
R.gInterpreter.Declare('''
void setup_gaus_tail(void){
auto gaustail = new TF1("gaustail",fit_gaus_tail, 0.,7.,4);
gaustail->SetParameters(1208, 3, 0.1, 1.);
gaustail->SetParNames("A","Mu","Sigma","tail");
std::cout << "The gaustail function is now setup. " << std::endl;
}
''')
R.setup_gaus_tail()

# Link the C++ fit_gaus_tail function to a TF1 so the fitting of the histogram works properly in Python
R.gInterpreter.Declare('''
void setup_gaus_tailn(void){
auto gaustailn = new TF1("gaustailn",fit_gaus_tailn, 0., 7.,4);
gaustailn->SetParameters(1208, 3, 0.1, 1.);
gaustailn->SetParNames("A","Mu","Sigma","tail");
std::cout << "The gaustailn function is now setup. " << std::endl;
}
''')
R.setup_gaus_tailn()

Welcome to JupyROOT 6.29/01
Recompile is not needed
The gaustail function is now setup. 
The gaustailn function is now setup. 


In [2]:
def make_ecal_snaphot(mini_dst, hist, opt=0):
    """Return a 2D histogram for the ECal, with the energy of the hits on the z-axis"""
    if hist is None:
        hist = R.TH2D("hist_ecal", "Ecal Hits", 50, -25.5, 24.5, 13, -6.5, 6.5)
    else:
        hist.Reset()
    # Fill the histogram by looping over the ECal hits
    for i in range(len(mini_dst.ecal_hit_index_x)):
        hist.Fill(mini_dst.ecal_hit_index_x[i], mini_dst.ecal_hit_index_y[i], mini_dst.ecal_hit_energy[i])
    return hist

In [4]:
ch = R.TChain("MiniDST")
file_peak_e = 3.742
ch.Add(f"/data/HPS/data/physrun2021/sim_2021/X/hpsForward_e-_{file_peak_e}GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
# ch.Add("/data/HPS/data/physrun2019/MC_RAW/hpsForward_e-_3.0GeV_z-7.5_x_SLIC-v06-00-01_QGSP_BERT_HPS-PhysicsRun2019-v2-4pt5_recon.root")           # sim_2019/hpsForward_e-_3.0GeV_*.root")
# ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
print(f"Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")
ch2 = R.TChain("MiniDST")
ch2.Add(f"/data/HPS/data/physrun2021/sim_2021/hpsForward_e-_{file_peak_e}GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
# ch.Add("/data/HPS/data/physrun2019/MC_RAW/hpsForward_e-_3.0GeV_z-7.5_x_SLIC-v06-00-01_QGSP_BERT_HPS-PhysicsRun2019-v2-4pt5_recon.root")           # sim_2019/hpsForward_e-_3.0GeV_*.root")
# ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
print(f"Chain2 N events loaded:  {ch2.GetEntries()/1e6:7.3f}M")

Number of events loaded:   1.832M
Chain2 N events loaded:    2.116M


In [4]:
df = R.RDataFrame(ch)
df2 = R.RDataFrame(ch2)
print("Available data names in Tuple:")
ColumnNames=df.GetColumnNames()
ll = 0
pr_colnames = [x for x in ColumnNames if str(x).startswith('')]
for nn in pr_colnames:
    if ll < len(nn):
        ll = len(nn)
for n in range(len(pr_colnames)):
    if n%4 == 0:
        print("")
    print(f"{str(pr_colnames[n]):{ll}s}",end="")

Available data names in Tuple:

ecal_cluster_energy           ecal_cluster_hits             ecal_cluster_nhits            ecal_cluster_seed_energy      
ecal_cluster_seed_index       ecal_cluster_seed_ix          ecal_cluster_seed_iy          ecal_cluster_time             
ecal_cluster_uncor_energy     ecal_cluster_uncor_hits       ecal_cluster_uncor_nhits      ecal_cluster_uncor_seed_energy
ecal_cluster_uncor_seed_index ecal_cluster_uncor_seed_ix    ecal_cluster_uncor_seed_iy    ecal_cluster_uncor_time       
ecal_cluster_uncor_x          ecal_cluster_uncor_y          ecal_cluster_uncor_z          ecal_cluster_x                
ecal_cluster_y                ecal_cluster_z                ecal_hit_energy               ecal_hit_index_x              
ecal_hit_index_y              ecal_hit_time                 event_number                  ext_trigger                   
hodo_cluster_energy           hodo_cluster_ix               hodo_cluster_iy               hodo_cluster_layer            


In [5]:
expected_peak = float(file_peak_e)
dfx = df.Define("n_cluster","ecal_cluster_energy.size()").Filter("n_cluster==1").Filter(f"ecal_cluster_seed_energy[0]>{expected_peak/2}")
dfx2 = df2.Define("n_cluster","ecal_cluster_energy.size()").Filter("n_cluster==1").Filter(f"ecal_cluster_seed_energy[0]>{expected_peak/2}")

In [6]:
min_e_histo = max(expected_peak-3.,0) # 0.5
max_e_histo = expected_peak*1.3 # 3.5

h_ecal_cluster_energy = dfx.Histo1D(("h_ecal_cluster_energy","Ecal Cluster Energy;E[GeV]",200,min_e_histo, max_e_histo),"ecal_cluster_energy")
h_ecal_cluster_energy_uncor = dfx.Histo1D(("h_ecal_cluster_uncor_energy","Ecal Un-corrected Cluster Energy;E[GeV]",200,min_e_histo, max_e_histo),"ecal_cluster_uncor_energy")
h_ecal_cluster_energy_lim = dfx.Histo1D(("h_ecal_cluster_energy","Ecal Cluster Energy;E[GeV]",200,min_e_histo, max_e_histo),"ecal_cluster_energy")
h2_ecal_cluster_energy = dfx2.Histo1D(("h_ecal_cluster_energy","Ecal Cluster Energy;E[GeV]",200,min_e_histo, max_e_histo),"ecal_cluster_energy")
h2_ecal_cluster_energy_uncor = dfx2.Histo1D(("h_ecal_cluster_uncor_energy","Ecal Un-corrected Cluster Energy;E[GeV]",200,min_e_histo, max_e_histo),"ecal_cluster_uncor_energy")
h2_ecal_cluster_energy_lim = dfx2.Histo1D(("h_ecal_cluster_energy","Ecal Cluster Energy;E[GeV]",200,min_e_histo, max_e_histo),"ecal_cluster_energy")

In [7]:
# R.gaustailn.SetParameters(20000, 4.9, 0.15, 5.);
cc0 = R.TCanvas("cc0","CC0",800, 600)
xax = h_ecal_cluster_energy.GetXaxis()

h_ecal_cluster_energy_uncor.SetStats(0)
h_ecal_cluster_energy.SetStats(0)
h_ecal_cluster_energy_uncor.SetLineColor(R.kGreen)
h_ecal_cluster_energy.SetLineColor(R.kBlue)
h2_ecal_cluster_energy_uncor.SetLineColor(R.kGreen+2)
h2_ecal_cluster_energy.SetLineColor(R.kCyan)
h_ecal_cluster_energy_uncor.SetTitle(f"Ecal Cluster Energies for e- primary of {file_peak_e} GeV")
h_ecal_cluster_energy_uncor.Draw()
h_ecal_cluster_energy.Draw("sameE")
h2_ecal_cluster_energy_uncor.Draw("same")
h2_ecal_cluster_energy.Draw("sameE")

#R.gaustailn.Draw("same")
fit_result = h_ecal_cluster_energy.Fit("gaus","S0","",h_ecal_cluster_energy.GetMean()*0.97,h_ecal_cluster_energy.GetMean()*1.08)
fit_result2 = h2_ecal_cluster_energy.Fit("gaus","S0","",h2_ecal_cluster_energy.GetMean()*0.97,h2_ecal_cluster_energy.GetMean()*1.08)

fit_fun = h_ecal_cluster_energy.GetFunction("gaus")
fit_fun.SetLineWidth(1)
fit_fun.SetLineColor(R.kRed+1)
fit_fun.Draw("same")
fit_fun2 = h2_ecal_cluster_energy.GetFunction("gaus")
fit_fun2.SetLineWidth(1)
fit_fun2.SetLineColor(R.kRed)
fit_fun2.Draw("same")

leg = R.TLegend(0.1,0.7,0.48,0.9)
leg.AddEntry(h_ecal_cluster_energy_uncor.GetPtr(),"Uncor Cluster E, no threshold")
leg.AddEntry(h2_ecal_cluster_energy_uncor.GetPtr(),"Uncor Cluster E, thres = 0.03")
leg.AddEntry(h_ecal_cluster_energy.GetPtr(),"Cluster E, no threshold")
leg.AddEntry(h2_ecal_cluster_energy.GetPtr(),"Cluster E, thres = 0.03")
leg.Draw()

text = R.TPaveText(0.13,0.4,0.45,0.6,"NDC")
text.SetTextFont(42)
text.SetTextAlign(12)
pt1 = text.AddText(f"Uncor Cluster E no thresh,   mean = {h_ecal_cluster_energy_uncor.GetMean():4.2f} GeV")
pt1.SetTextColor(R.kGreen)
pt2 = text.AddText(f"Uncor Cluster E thresh=0.03, mean = {h2_ecal_cluster_energy_uncor.GetMean():4.2f} GeV")
pt2.SetTextColor(R.kGreen+2)
pt3 = text.AddText(f"Cor Cluster no thresh,       mean = {h_ecal_cluster_energy.GetMean():4.2f} GeV")
pt3.SetTextColor(R.kBlue)
pt4 = text.AddText(f"Cor Cluster E thresh=0.03,   mean = {h2_ecal_cluster_energy.GetMean():4.2f} GeV")
pt4.SetTextColor(R.kCyan)
pt5 = text.AddText(f"Cor Cluster no thresh,       fit #mu = {fit_result.Parameter(1):4.2f} GeV")
pt5.SetTextColor(R.kRed+1)
pt6 = text.AddText(f"Cor Cluster E thresh=0.03,   fit #mu = {fit_result2.Parameter(1):4.2f} GeV")
pt6.SetTextColor(R.kRed)

text.Draw()


cc0.Draw()
cc0.SaveAs(f"ECal_energy_distribution_{file_peak_e}GeV_2021.pdf")

 FCN=966.538 FROM MIGRAD    STATUS=CONVERGED      76 CALLS          77 TOTAL
                     EDM=1.04698e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.91188e+04   4.88976e+01   5.60353e-01  -1.09662e-05
   2  Mean         4.74492e+00   3.07857e-04   4.40581e-06   4.01057e+00
   3  Sigma        1.29520e-01   2.99722e-04   9.20590e-06   9.16356e-01
 FCN=923.133 FROM MIGRAD    STATUS=CONVERGED      64 CALLS          65 TOTAL
                     EDM=6.52844e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.95424e+04   4.90279e+01   5.53348e-01  -1.06843e-05
   2  Mean         4.68538e+00   2.94842e-04   4.12037e-06   3.65849e+00
   3  Sigma        1.28264e-01   

Info in <TCanvas::Print>: pdf file ECal_energy_distribution_4.5GeV.pdf has been created
